<a href="https://colab.research.google.com/github/alexvanhalen/alexvanhalen.github.io/blob/master/Test_Rinex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

先安裝georinex，直接pip install即可，不用clone git，github上面放的是beta版。

In [1]:
!pip install georinex

     |████████████████████████████████| 6.5MB 4.7MB/s 
  Created wheel for georinex: filename=georinex-1.11.0-cp36-none-any.whl size=40195 sha256=b4fa09df84fa493ea96eaa3900844316e0c717029dee84ba25876a189a5b311c
  Stored in directory: /root/.cache/pip/wheels/c2/05/21/8c5c752b30b65770743e0c41c2dcff731371e39c07172106c1
Successfully built georinex


安裝完之後，可以直接import，取名為gr。gr.load()可以將rinex檔案load進來，程式本身自己會判斷是observation (副檔名通常是.xxo)，還是navigation (副檔名通常是.xxn)。因為這邊要計算衛星軌道，所以先讀取navigation檔案。

In [5]:
import georinex as gr
nav = gr.load('test.19n')

print(nav)

<xarray.Dataset>
Dimensions:           (sv: 12, time: 2)
Coordinates:
  * sv                (sv) <U3 'G01' 'G03' 'G06' 'G07' ... 'G22' 'G28' 'G30'
  * time              (time) datetime64[ns] 2019-04-22T09:59:44 2019-04-22T10:00:00
Data variables:
    SVclockBias       (time, sv) float64 -0.0002073 nan nan ... 0.0007676 nan
    SVclockDrift      (time, sv) float64 -8.299e-12 nan nan ... 1.137e-13 nan
    SVclockDriftRate  (time, sv) float64 0.0 nan nan nan nan ... nan 0.0 0.0 nan
    IODE              (time, sv) float64 16.0 nan nan nan ... nan 73.0 34.0 nan
    Crs               (time, sv) float64 -30.06 nan nan nan ... 13.44 30.0 nan
    DeltaN            (time, sv) float64 4.017e-09 nan nan ... 4.049e-09 nan
    M0                (time, sv) float64 0.9865 nan nan nan ... 2.383 -2.305 nan
    Cuc               (time, sv) float64 -1.613e-06 nan nan ... 1.706e-06 nan
    Eccentricity      (time, sv) float64 0.008678 nan nan ... 0.01909 nan
    Cus               (time, sv) float64 1.084e

檔案讀取進來之後，放在nav這個物件，其結構如上所示。每一個變數的定義可以參考：https://ascelibrary.org/doi/pdf/10.1061/9780784411506.ap03
可以先用nav.sv看看目前有哪幾顆衛星。

In [9]:
print(nav.sv)

<xarray.DataArray 'sv' (sv: 12)>
array(['G01', 'G03', 'G06', 'G07', 'G08', 'G11', 'G17', 'G18', 'G19', 'G22',
       'G28', 'G30'], dtype='<U3')
Coordinates:
  * sv       (sv) <U3 'G01' 'G03' 'G06' 'G07' 'G08' ... 'G19' 'G22' 'G28' 'G30'


可看到有G01, G03, G06, ...等12顆衛星。接下來使用nav.sel( )選擇特定衛星，例如：

In [10]:
G1 = nav.sel(sv='G01')
print(G1)

<xarray.Dataset>
Dimensions:           (time: 2)
Coordinates:
    sv                <U3 'G01'
  * time              (time) datetime64[ns] 2019-04-22T09:59:44 2019-04-22T10:00:00
Data variables:
    SVclockBias       (time) float64 -0.0002073 nan
    SVclockDrift      (time) float64 -8.299e-12 nan
    SVclockDriftRate  (time) float64 0.0 nan
    IODE              (time) float64 16.0 nan
    Crs               (time) float64 -30.06 nan
    DeltaN            (time) float64 4.017e-09 nan
    M0                (time) float64 0.9865 nan
    Cuc               (time) float64 -1.613e-06 nan
    Eccentricity      (time) float64 0.008678 nan
    Cus               (time) float64 1.084e-05 nan
    sqrtA             (time) float64 5.154e+03 nan
    Toe               (time) float64 1.224e+05 nan
    Cic               (time) float64 -4.098e-08 nan
    Omega0            (time) float64 -2.352 nan
    Cis               (time) float64 9.686e-08 nan
    Io                (time) float64 0.9752 nan
    Crc   

將G01衛星的導航資料取出來，存放在G1這個物件。G1有兩個columns，第一個column有資料，第二個column沒有資料 ( 顯示為nan)。
參考https://ascelibrary.org/doi/pdf/10.1061/9780784411506.ap0，假設要計算delta_t_sv：

In [13]:
print(G1.IODC)
t = 1 #這邊要用t-toc代入，方便起見，先令為1。toc就是上面的IODC
delta_t_sv = G1.SVclockBias + G1.SVclockDrift*t + G1.SVclockDriftRate * t**2
print(delta_t_sv)

<xarray.DataArray 'IODC' (time: 2)>
array([16., nan])
Coordinates:
    sv       <U3 'G01'
  * time     (time) datetime64[ns] 2019-04-22T09:59:44 2019-04-22T10:00:00
<xarray.DataArray (time: 2)>
array([-0.000207,       nan])
Coordinates:
    sv       <U3 'G01'
  * time     (time) datetime64[ns] 2019-04-22T09:59:44 2019-04-22T10:00:00


To be continued....